# AI Intern Challenge


In [ ]:
# download all dependendencies, comment out if already installed
!pip install vllm
!pip install langchain faiss-cpu
!pip install -U langchain-community
!pip install huggingface_hub
!pip install langchain_community
!pip install chromadb
!pip install langchainhub
!pip install beautifulsoup4
!pip install requests
!pip install python-docx
!pip install -U sentence-transformers
!pip install unsloth
!pip install datasets
!pip install peft
!pip install transformers>=4.36.0
!pip install accelerate>=0.21.0
!pip install bitsandbytes>=0.41.0

In [ ]:
# all imports
from unsloth import FastLanguageModel
from huggingface_hub import notebook_login
import json
from typing import Dict
from vllm import LLM, SamplingParams
import re
import requests
import time
import os
from google.colab import drive # comment out if you're not using colab

# for Fine-Tuning
import pandas as pd
from unsloth import FastLanguageModel
import torch
from datasets import Dataset
from peft import AutoPeftModelForCausalLM

# For RAG
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
import transformers
from concurrent.futures import ThreadPoolExecutor, as_completed

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-12 03:05:44 [__init__.py:239] Automatically detected platform cuda.


In [ ]:
# Enter access tokens, remove if saved in secrets
notebook_login() # access token for hugging face
os.environ['OPENAI_API_KEY'] = 'Your_OpenAI_Key' # your Open AI key

In [ ]:
# Load All Data

drive.mount('/content/drive') # comment out if you're not using colab

def load_data(file_path: str) -> Dict:
    """
    Load the JSON data from a file.

    Args:
    file_path (str): Path to the JSON file.

    Returns:
    Dict: Loaded JSON data.
    """
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# these variables
train_data_filepath = "/content/drive/MyDrive/airaChallenge/data/train_data.json"
val_data_filepath = "/content/drive/MyDrive/airaChallenge/data/val_data.json"

train_data = load_data(train_data_filepath)
val_data = load_data(val_data_filepath)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# define evaluate_model function

def evaluate_model(data: Dict, predictions: Dict) -> float:
    """
    Evaluate the model's performance on a dataset.

    Args:
    data (Dict): The dataset to evaluate on (can be validation or test set).
    predictions (Dict): The model's predictions for the dataset.

    Returns:
    float: Accuracy of the model.
    """
    correct = 0
    total = len(data)

    for question_id, question_data in enumerate(data):
        if question_id in predictions:
            if predictions[question_id] == question_data['answer']:
                correct += 1

    accuracy = correct / total
    return accuracy


## Part A: Basic Inference Engine

Usage
1. Load the Model
2. Define `inference_naive(data, llm, sampling_params)`
3. Evaluate

In [ ]:
# Load the model

llm = LLM(
    model="meta-llama/Llama-3.2-3B-Instruct",
    dtype="float16",
    gpu_memory_utilization=0.65,
    max_model_len=4000
)

sampling_params = SamplingParams(temperature=0.01, max_tokens=22)


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

WARNING 04-10 21:59:18 [config.py:2704] Casting torch.bfloat16 to torch.float16.
INFO 04-10 21:59:33 [config.py:600] This model supports multiple tasks: {'generate', 'embed', 'score', 'classify', 'reward'}. Defaulting to 'generate'.
WARNING 04-10 21:59:33 [arg_utils.py:1708] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 04-10 21:59:33 [llm_engine.py:242] Initializing a V0 LLM engine (v0.8.3) with config: model='meta-llama/Llama-3.2-3B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-3B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decodin

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 04-10 22:02:57 [loader.py:447] Loading weights took 31.68 seconds
INFO 04-10 22:02:57 [model_runner.py:1146] Model loading took 6.0160 GiB and 199.589262 seconds
INFO 04-10 22:03:00 [worker.py:267] Memory profiling takes 2.15 seconds
INFO 04-10 22:03:00 [worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.65) = 9.58GiB
INFO 04-10 22:03:00 [worker.py:267] model weights take 6.02GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 1.19GiB; the rest of the memory reserved for KV Cache is 2.33GiB.
INFO 04-10 22:03:01 [executor_base.py:112] # cuda blocks: 1360, # CPU blocks: 2340
INFO 04-10 22:03:01 [executor_base.py:117] Maximum concurrency for 4000 tokens per request: 5.44x
INFO 04-10 22:03:06 [model_runner.py:1456] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. 

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:44<00:00,  1.28s/it]

INFO 04-10 22:03:51 [model_runner.py:1598] Graph capturing finished in 45 secs, took 0.19 GiB
INFO 04-10 22:03:51 [llm_engine.py:448] init engine (profile, create kv cache, warmup model) took 53.64 seconds


### Inference Naive (Prompt Engineering + Experimenting with Sampling Prompts)

In [ ]:
def inference_naive(data, llm, sampling_params):
    """
    Use Llama 3.2-3B instruct model directly via the vLLM Python API
    to answer questions from the TeleQnA dataset.

    Args:
        data (list): The dataset (list of question dicts).
        llm (vllm.LLM): An already-instantiated LLM object from vLLM.
        sampling_params (vllm.SamplingParams): Parameters for generation.

    Returns:
        Dict[int, str]: A dictionary mapping question_id to predicted answer.
    """

    # Prepare prompts for all questions
    prompts = []
    question_ids = []

    print(f"Preparing prompts for {len(data)} questions...")

    for question_id, question_data in enumerate(data):
        # Build a prompt for each question with all its options.
        prompt = (
            "Answer the following multiple-choice question about telecommunications. "
            "Only respond with exactly one option (e.g., 'option 1', 'option 2', etc.) "
            "without any additional text.\n\n"
            "Example 1:\n"
            "Question: What is the capital of France?\n"
            "option 1: Berlin\n"
            "option 2: Madrid\n"
            "option 3: Paris\n"
            "option 4: Rome\n"
            "option 5: Oslo\n"
            "Answer: option 3\n\n"
            "Example 2:\n"
            "Question: What does HTTPS stand for?\n"
            "option 1: Hypertext Transfer Protocol Secure\n"
            "option 2: Hypertext Transfer Protocol Standard\n"
            "option 3: Hypertext Transfer Protocol System\n"
            "option 4: Hypertext Transfer Protocol Specification\n"
            "option 5: Hypertext Transfer Protocol\n"
            "Answer: option 1\n\n"
            f"Now, answer the question below in the same format:\n"
            f"Question: {question_data['question']}\n"
        )

        # Add all available options
        for option_key in sorted(k for k in question_data if k.startswith("option ")):
            prompt += f"{option_key}: {question_data[option_key]}\n"

        prompt += "\nAnswer: "

        prompts.append(prompt)
        question_ids.append(question_id)

    # Process in batches for efficiency
    batch_size = 32
    predictions = {}

    print(f"Running inference in batches of {batch_size}...")
    for i in range(0, len(prompts), batch_size):
        batch_prompts = prompts[i : i + batch_size]
        batch_ids = question_ids[i : i + batch_size]
        print(f"Processing batch {i//batch_size + 1}/{(len(prompts) + batch_size - 1)//batch_size}")

        # Generate with vLLM
        outputs = llm.generate(
                            prompts=batch_prompts,
                            sampling_params=sampling_params  # crucial keyword
                        )
        # Process the outputs
        for output, q_id in zip(outputs, batch_ids):
            # Get generated text
            generated_text = output.outputs[0].text.strip().lower() if output.outputs else ""

            # Try to match using regex to capture both "option" and just a digit
            match = re.search(r"option\s*(\d)", generated_text)
            if not match:
                # If no "option" found, check if there's just a standalone digit
                match = re.search(r"^\D*(\d)\D*$", generated_text)

            if match:
                option_number = match.group(1)
                predictions[q_id] = f"option {option_number}"
            else:
                predictions[q_id] = "option 1"  # default fallback


    print(f"Inference completed for {len(predictions)} questions.")
    return predictions


In [ ]:
start = time.time()
predictions = inference_naive(val_data, llm, sampling_params)
end = time.time()
print(f"Inference took {end - start} seconds.")
runtime_naive = end - start

Preparing prompts for 2000 questions...
Running inference in batches of 32...
Processing batch 1/63


Processed prompts: 100%|██████████| 32/32 [00:02<00:00, 11.89it/s, est. speed input: 2939.89 toks/s, output: 27.53 toks/s]


Processing batch 2/63


Processed prompts: 100%|██████████| 32/32 [00:02<00:00, 11.99it/s, est. speed input: 2962.82 toks/s, output: 32.98 toks/s]


Processing batch 3/63


...

Processing batch 62/63


Processed prompts: 100%|██████████| 32/32 [00:02<00:00, 12.18it/s, est. speed input: 2993.31 toks/s, output: 32.76 toks/s]


Processing batch 63/63


Processed prompts: 100%|██████████| 16/16 [00:01<00:00, 11.47it/s, est. speed input: 2837.04 toks/s, output: 30.15 toks/s]

Inference completed for 2000 questions.
Inference took 180.6603980064392 seconds.


In [ ]:
# Evaluate the predictions
accuracy = evaluate_model(val_data, predictions)
print(f"Model accuracy: {accuracy * 100:.2f}%")
print(f"Inference took {runtime_naive} seconds.")

Model accuracy: 63.05%
Inference took 180.6603980064392 seconds.


## Part B: Fine-Tuning the LLM

Instructions to Run:
1. **Restart Session** to free up System RAM
2. Import dependencies
3. Load data
4. Train model (alter parameters, if required)
5. Restart Session again as notebook environments like Colab or Jupyter the state can become inconsistent during a long session.
6. Define `inference_fine_tuned(data, model_path, batch_size=32)`
7. Evaluate Checkpoints and choose the best model

In [ ]:
# all imports
from unsloth import FastLanguageModel
from datasets import Dataset
import json
import pandas as pd
from transformers import TrainingArguments, Trainer
from trl import SFTTrainer
import re
from google.colab import drive # comment out if you're not using colab

from huggingface_hub import notebook_login
from typing import Dict
from vllm import LLM, SamplingParams
import time
import os
import torch
from peft import AutoPeftModelForCausalLM
import gc

# For RAG
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
import transformers
from concurrent.futures import ThreadPoolExecutor, as_completed

<ipython-input-2-30b311ced9a6>:10: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-13 04:21:43 [__init__.py:239] Automatically detected platform cuda.


In [ ]:
# Load All Data

drive.mount('/content/drive') # comment out if you're not using colab

def load_data(file_path: str) -> Dict:
    """
    Load the JSON data from a file.

    Args:
    file_path (str): Path to the JSON file.

    Returns:
    Dict: Loaded JSON data.
    """
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# these variables
train_data_filepath = "/content/drive/MyDrive/airaChallenge/data/train_data.json"
val_data_filepath = "/content/drive/MyDrive/airaChallenge/data/val_data.json"

train_data = load_data(train_data_filepath)
val_data = load_data(val_data_filepath)
results = load_data("/content/drive/MyDrive/airaChallenge/results.json")

# define evaluate_model function

def evaluate_model(data: Dict, predictions: Dict) -> float:
    """
    Evaluate the model's performance on a dataset.

    Args:
    data (Dict): The dataset to evaluate on (can be validation or test set).
    predictions (Dict): The model's predictions for the dataset.

    Returns:
    float: Accuracy of the model.
    """
    correct = 0
    total = len(data)

    for question_id, question_data in enumerate(data):
        if question_id in predictions:
            if predictions[question_id] == question_data['answer']:
                correct += 1

    accuracy = correct / total
    return accuracy



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

def prepare_dataset_for_finetuning(data):
    """Convert TeleQnA data into instruction format for fine-tuning"""
    formatted_data = []

    for item in data:
        # Create prompt with question and options
        question = item['question']
        options_text = ""
        for option_key in sorted(k for k in item if k.startswith("option ")):
            options_text += f"{option_key}: {item[option_key]}\n"

        prompt = f"You're a telemcommunications expert. Here's a multiple-choice question about telecommunications' {item['category'].lower() if item['category'] else ''}. Consider technical details carefully. Only respond with exactly one option without additional text.\n\nQuestion: {question}\n{options_text}"

        # The completion is just the answer option
        completion = item['answer']

        formatted_data.append({
            "text": f"<s>[INST] {prompt} [/INST] {completion} </s>"
        })



    return  Dataset.from_pandas(pd.DataFrame(formatted_data))

In [ ]:
start = time.time()
train_dataset = prepare_dataset_for_finetuning(train_data)
val_dataset = prepare_dataset_for_finetuning(val_data)

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="meta-llama/Llama-3.2-3B-Instruct",
    max_seq_length=2048,
    dtype=torch.float16,
    load_in_4bit=True,  # Use 4-bit quantization to fit on T4
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.2. vLLM: 0.8.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
# Add LoRA adapters for efficient fine-tuning
model = FastLanguageModel.get_peft_model(
    model,
    r=32,              # Rank
    lora_alpha=64,     # Alpha parameter for LoRA
    lora_dropout=0.05,  # Dropout probability for LoRA
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj"],
)


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.3.19 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
training_args = TrainingArguments(
    output_dir="./telecom_finetuned",
    # Instead of setting num_train_epochs=6, set max_steps directly
    max_steps=800,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    warmup_ratio=0.05,
    lr_scheduler_type="cosine",
    logging_steps=25,
    save_strategy="steps",
    save_steps=50,                # Save more frequently to find the optimal point
    optim="adamw_torch",
    fp16=True,
    bf16=False,
    weight_decay=0.01,
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    args=training_args,
)
os.environ['UNSLOTH_RETURN_LOGITS'] = '1'
trainer.train()
end = time.time()
print(f"Training took {end - start} seconds.")
fine_tuning_train_time = end - start

<ipython-input-9-fd88de9c8b6a>:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Converting train dataset to ChatML (num_proc=2):   0%|          | 0/6000 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/6000 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/6000 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 6,000 | Num Epochs = 3 | Total steps = 800
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 48,627,712/3,000,000,000 (1.62% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: shreyasudan2211 (shreyasudan2211-university-of-california) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
25,2.033600
50,1.060700
75,1.036800
100,1.006700
125,0.994400
150,0.984000
175,0.974900
200,0.974500
225,0.967400
250,0.973200


Training took 3240.2419049739838 seconds.


🚨**STOP: Restart the Session**🚨

To avoid memory leaks from previous model loads, likely a runtime environment issue, restart the notebook. This happens sometimes in notebook environments like Colab or Jupyter where the state can become inconsistent during a long session.



In [ ]:
# all imports
from unsloth import FastLanguageModel
from datasets import Dataset
import json
import pandas as pd
from transformers import TrainingArguments, Trainer, AutoTokenizer
from trl import SFTTrainer
import re
from google.colab import drive # comment out if you're not using colab

from huggingface_hub import notebook_login
from typing import Dict
from vllm import LLM, SamplingParams
import time
import os
import torch
from peft import AutoPeftModelForCausalLM

# For RAG
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
import transformers
from concurrent.futures import ThreadPoolExecutor, as_completed

# Load All Data

drive.mount('/content/drive') # comment out if you're not using colab

def load_data(file_path: str) -> Dict:
    """
    Load the JSON data from a file.

    Args:
    file_path (str): Path to the JSON file.

    Returns:
    Dict: Loaded JSON data.
    """
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# these variables
train_data_filepath = "/content/drive/MyDrive/airaChallenge/data/train_data.json"
val_data_filepath = "/content/drive/MyDrive/airaChallenge/data/val_data.json"

train_data = load_data(train_data_filepath)
val_data = load_data(val_data_filepath)

# define evaluate_model function

def evaluate_model(data: Dict, predictions: Dict) -> float:
    """
    Evaluate the model's performance on a dataset.

    Args:
    data (Dict): The dataset to evaluate on (can be validation or test set).
    predictions (Dict): The model's predictions for the dataset.

    Returns:
    float: Accuracy of the model.
    """
    incorrect = {}
    correct = 0
    total = len(data)

    for question_id, question_data in enumerate(data):
        if question_id in predictions:
            if predictions[question_id] == question_data['answer']:
                correct += 1
            # else:
              # print(f"Incorrect: {question_data['question']}")
              # print(f"Correct answer: {question_data['category']}")

    accuracy = correct / total
    return accuracy


<ipython-input-1-3b1cb38a9c14>:10: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-13 08:49:28 [__init__.py:239] Automatically detected platform cuda.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def inference_fine_tuned(data, model_path, batch_size=32):
    """Use fine-tuned model with batching for faster inference"""

    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoPeftModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.float16,
        device_map="auto",
    )

    predictions = {}

    # Process in batches
    for i in range(0, len(data), batch_size):
        batch_data = data[i:i+batch_size]
        batch_prompts = []
        batch_ids = []

        for question_id, question_data in enumerate(batch_data, start=i):
            question = question_data['question']
            options_text = ""
            for option_key in sorted(k for k in question_data if k.startswith("option ")):
                options_text += f"{option_key}: {question_data[option_key]}\n"

            prompt = f"""You're a telemcommunications expert. Here's a multiple-choice question about telecommunications' {question_data['category'].lower() if question_data['category'] else ''}. Consider technical details carefully. Only respond with exactly one option without additional text, for example,

Example:
Question: What does HTTPS stand for?
option 1: Hypertext Transfer Protocol Secure
option 2: Hypertext Transfer Protocol Standard
option 3: Hypertext Transfer Protocol System
option 4: Hypertext Transfer Protocol Specification
option 5: Hypertext Transfer Protocol
Answer: option 1

Now, answer the question below in the same format:
Question: {question}
{options_text}[/INST]"""

            batch_prompts.append(prompt)
            batch_ids.append(question_id)

        # Tokenize batch
        inputs = tokenizer(batch_prompts, padding=True, padding_side='left', return_tensors="pt").to(model.device)

        # Generate for whole batch
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens=20,
                temperature=0.01,
                do_sample=False
            )

        # Process batch outputs
        for output, q_id, prompt in zip(outputs, batch_ids, batch_prompts):
            generated_text = tokenizer.decode(output, skip_special_tokens=True)

            # Remove the prompt from the generated text
            if prompt in generated_text:
                generated_text = generated_text[len(prompt):].strip().lower()
            else:
                generated_text = generated_text.strip().lower()

            # Extract answer
            match = re.search(r"option\s*(\d)", generated_text)
            if not match:
                match = re.search(r"^\D*(\d)\D*$", generated_text)

            if match:
                option_number = match.group(1)
                predictions[q_id] = f"option {option_number}"
            else:
                predictions[q_id] = "option 1"  # default fallback

    return predictions

In [ ]:
def evaluate_all_checkpoints(checkpoints, val_data):
    results = {}

    for ckpt in checkpoints:
        start = time.time()
        # Run inference and evaluation
        predictions = inference_fine_tuned(val_data, ckpt)
        end = time.time()
        print(f"Inference took {end - start} seconds.")
        runtime_batch = end - start
        accuracy = evaluate_model(val_data, predictions)

        results[ckpt] = {'accuracy' : accuracy, 'time' : runtime_batch}
        print(f"Checkpoint {ckpt}: accuracy = {accuracy * 100:.2f}%")

    return results

In [ ]:
# Test each checkpoint
checkpoints = [
    "/content/telecom_finetuned/checkpoint-400",
    "/content/telecom_finetuned/checkpoint-500",
    "/content/telecom_finetuned/checkpoint-600",
    "/content/telecom_finetuned/checkpoint-700",
    "/content/telecom_finetuned/checkpoint-800",
]

evaluate_all_checkpoints(checkpoints, val_data)

Inference took 1663.2670772075653 seconds.
Checkpoint /content/telecom_finetuned/checkpoint-400: accuracy = 68.65%
Inference took 1654.464478969574 seconds.
Checkpoint /content/telecom_finetuned/checkpoint-500: accuracy = 70.30%
Inference took 1653.3009474277496 seconds.
Checkpoint /content/telecom_finetuned/checkpoint-600: accuracy = 69.05%
Inference took 1655.3874034881592 seconds.
Checkpoint /content/telecom_finetuned/checkpoint-700: accuracy = 69.25%
Inference took 1658.7776672840118 seconds.
Checkpoint /content/telecom_finetuned/checkpoint-800: accuracy = 69.40%


{'/content/telecom_finetuned/checkpoint-400': {'accuracy': 0.6865,
  'time': 1663.2670772075653},
 '/content/telecom_finetuned/checkpoint-500': {'accuracy': 0.703,
  'time': 1654.464478969574},
 '/content/telecom_finetuned/checkpoint-600': {'accuracy': 0.6905,
  'time': 1653.3009474277496},
 '/content/telecom_finetuned/checkpoint-700': {'accuracy': 0.6925,
  'time': 1655.3874034881592},
 '/content/telecom_finetuned/checkpoint-800': {'accuracy': 0.694,
  'time': 1658.7776672840118}}

In [ ]:
# Test each checkpoint
checkpoints = [
    "/content/telecom_finetuned_model_increased_epochs/checkpoint-100",
    "/content/telecom_finetuned_model_increased_epochs/checkpoint-150",
    "/content/telecom_finetuned_model_increased_epochs/checkpoint-200",
    "/content/telecom_finetuned_model_increased_epochs/checkpoint-250",
    "/content/telecom_finetuned_model_increased_epochs/checkpoint-300",
    "/content/telecom_finetuned_model_increased_epochs/checkpoint-350",
    "/content/telecom_finetuned_model_increased_epochs/checkpoint-400",
]

evaluate_all_checkpoints(checkpoints, val_data)

Checkpoint /content/telecom_finetuned_model/checkpoint-100: accuracy = 68.90%
Checkpoint /content/telecom_finetuned_model/checkpoint-150: accuracy = 68.75%
Checkpoint /content/telecom_finetuned_model/checkpoint-200: accuracy = 70.30%
Checkpoint /content/telecom_finetuned_model/checkpoint-250: accuracy = 69.85%
Checkpoint /content/telecom_finetuned_model/checkpoint-300: accuracy = 70.90%
Checkpoint /content/telecom_finetuned_model/checkpoint-350: accuracy = 71.10%
Checkpoint /content/telecom_finetuned_model/checkpoint-400: accuracy = 71.25%


In [ ]:
# Pick the best model, this may differ for your run
best_model = "/content/telecom_finetuned/checkpoint-400"

In [ ]:
start = time.time()
predictions = inference_fine_tuned(val_data, best_model)
end = time.time()
print(f"Inference took {end - start} seconds.")
runtime_batch = end - start
accuracy = evaluate_model(val_data, predictions)
print(f"Model accuracy: {accuracy * 100:.2f}%")

Inference took 1248.202805519104 seconds.
Model accuracy: 72.25%


### Appendix: RAG Implementation on the Fine-Tuned Model

1. Let's restart the session to free up our GPU and RAM
2. Import dependencies
3. Set up your OpenAI API Key
4. Extract information from the Wireless Documents [3GPP](https://www.3gpp.org/ftp/Specs/archive/38_series/38.201/) after storing the files locally
5. Extract information from training dataset
6. Prepare the documents into chunks
7. Create a vectore store
8. Run `inference_with_rag`

In [ ]:
from unsloth import FastLanguageModel
import os
import re
import json
import numpy as np
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
import torch
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from google.colab import drive

# all imports
from datasets import Dataset
import pandas as pd
from transformers import TrainingArguments, Trainer
from trl import SFTTrainer
from google.colab import drive # comment out if you're not using colab

from huggingface_hub import notebook_login
from typing import Dict
from vllm import LLM, SamplingParams
import time
import os
import torch
from peft import AutoPeftModelForCausalLM

# For RAG
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
import transformers
from concurrent.futures import ThreadPoolExecutor, as_completed

# Load All Data

drive.mount('/content/drive') # comment out if you're not using colab

def load_data(file_path: str) -> Dict:
    """
    Load the JSON data from a file.

    Args:
    file_path (str): Path to the JSON file.

    Returns:
    Dict: Loaded JSON data.
    """
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# these variables
train_data_filepath = "/content/drive/MyDrive/airaChallenge/data/train_data.json"
val_data_filepath = "/content/drive/MyDrive/airaChallenge/data/val_data.json"

train_data = load_data(train_data_filepath)
val_data = load_data(val_data_filepath)
results = load_data("/content/drive/MyDrive/airaChallenge/results.json")

# define evaluate_model function

def evaluate_model(data: Dict, predictions: Dict) -> float:
    """
    Evaluate the model's performance on a dataset.

    Args:
    data (Dict): The dataset to evaluate on (can be validation or test set).
    predictions (Dict): The model's predictions for the dataset.

    Returns:
    float: Accuracy of the model.
    """
    correct = 0
    total = len(data)

    for question_id, question_data in enumerate(data):
        if question_id in predictions:
            if predictions[question_id] == question_data['answer']:
                correct += 1

    accuracy = correct / total
    return accuracy

# Enter access tokens, remove if saved in secrets
notebook_login() # access token for hugging face
os.environ['OPENAI_API_KEY'] = 'Your_OpenAI_Key' # your Open AI key

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-13 09:04:08 [__init__.py:239] Automatically detected platform cuda.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 1. Reading the Wireless Documents at https://www.3gpp.org/ftp/Specs/archive/38_series/38.201/

def prepare_3gpp_documents():
    """Process 3GPP documents from Google Drive or use sample documents if not found"""
    print("Processing 3GPP documents...")
    from langchain.schema import Document

    # Try to import required libraries
    try:
        import os
        from docx import Document as DocxDocument
        docx_available = True
    except ImportError:
        print("Warning: python-docx not available. Installing...")
        !pip install python-docx
        import os
        from docx import Document as DocxDocument
        docx_available = True

    documents = []

    # Try to mount Google Drive if not already mounted
    try:
        from google.colab import drive
        drive_already_mounted = os.path.exists('/content/drive')
        if not drive_already_mounted:
            print("Mounting Google Drive...")
            drive.mount('/content/drive')

        # Try multiple possible paths where documents might be stored
        possible_paths = [
            "/content/drive/MyDrive/airaChallenge/wireless_documents",
            "/content/drive/My Drive/airaChallenge/wireless_documents",
            "./wireless_documents",
            "/content/wireless_documents"
        ]

        found_docs = False
        docs_dir = None

        for path in possible_paths:
            if os.path.exists(path):
                docs_dir = path
                found_docs = True
                print(f"Found documents directory at: {docs_dir}")
                break

        if found_docs:
            # List all document files in the directory
            doc_count = 0
            for filename in os.listdir(docs_dir):
                if filename.endswith(".doc") or filename.endswith(".docx") or filename.endswith(".txt") or filename.endswith(".pdf"):
                    try:
                        filepath = os.path.join(docs_dir, filename)

                        # Different handling based on file type
                        if filename.endswith(".docx"):
                            try:
                                doc = DocxDocument(filepath)
                                content = "\n".join([para.text for para in doc.paragraphs if para.text])
                            except Exception as e:
                                print(f"Error processing {filename} with docx: {e}")
                                content = f"Document {filename} content extraction failed"
                        elif filename.endswith(".txt"):
                            with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
                                content = f.read()
                        else:
                            # For .doc and .pdf, just note we found it
                            content = f"Document {filename} found but format not fully supported for extraction."

                        # Create a document object
                        documents.append(Document(
                            page_content=content,
                            metadata={"source": filepath, "title": filename}
                        ))
                        doc_count += 1
                        if doc_count % 10 == 0:  # Print status every 10 docs
                            print(f"Processed {doc_count} documents so far...")
                    except Exception as e:
                        print(f"Error processing {filename}: {e}")

            print(f"Successfully processed {doc_count} documents from {docs_dir}")
        else:
            print("No documents directory found in common locations. Using sample documents.")
            documents = create_sample_telecom_documents()
    except Exception as e:
        print(f"Error accessing Google Drive: {e}")
        print("Using sample telecom documents instead.")
        documents = create_sample_telecom_documents()

    # If no documents were processed, use sample documents
    if not documents:
        print("No documents were processed. Using sample telecom knowledge...")
        documents = create_sample_telecom_documents()

    print(f"Total processed documents: {len(documents)}")
    return documents

def create_sample_telecom_documents():
    """Create sample telecom documents with relevant 3GPP information"""
    documents = []

    # Sample document from the 3GPP TS 38.201 you shared
    documents.append(Document(
        page_content="""
        3GPP TS 38.201 V0.0.0 (2017-05)
        Technical Specification 3rd Generation Partnership Project;
        Technical Specification Group Radio Access Network;
        NR; Physical layer; General description (Release 15)

        The present document describes a general description of the physical layer of 5G-NR.

        1 Scope
        The present document describes a general description of the physical layer of 5G-NR.

        2 References
        The following documents contain provisions which, through reference in this text, constitute provisions of the present document.
        [1] 3GPP TR 21.905: "Vocabulary for 3GPP Specifications"
        [2] 3GPP TS 38.202: "NR; Services provided by the physical layer"
        [3] 3GPP TS 38.211: "NR; Physical channels and modulation"
        [4] 3GPP TS 38.212: "NR; Multiplexing and channel coding"
        [5] 3GPP TS 38.xxx: "NR; Physical layer procedures for control channels"
        [6] 3GPP TS 38.yyy: "NR; Physical layer procedures for data channels"
        [7] 3GPP TS 38.215: "NR; Physical layer measurements"
        """,
        metadata={"source": "sample_data", "title": "38.201-NR-Physical-layer-General-description.doc"}
    ))

    # Add more sample documents for key 5G concepts
    documents.append(Document(
        page_content="""
        5G NR (New Radio) Key Concepts:

        Frequency Ranges:
        - FR1 (Sub-6GHz): 450 MHz - 6000 MHz
        - FR2 (mmWave): 24.25 GHz - 52.6 GHz

        Subcarrier Spacing Options:
        - 15 kHz, 30 kHz, 60 kHz, 120 kHz, 240 kHz

        Numerology:
        - Based on subcarrier spacing
        - μ=0 (15 kHz), μ=1 (30 kHz), μ=2 (60 kHz), μ=3 (120 kHz), μ=4 (240 kHz)

        Channel Coding:
        - LDPC (Low-Density Parity Check) for data channels
        - Polar codes for control channels

        Multiple Access:
        - CP-OFDM (Cyclic Prefix Orthogonal Frequency Division Multiplexing)

        MIMO Capabilities:
        - Massive MIMO
        - Multi-user MIMO
        - Beamforming techniques
        """,
        metadata={"source": "sample_data", "title": "5G-NR-Key-Concepts.doc"}
    ))

    # Add document about 5G network architecture
    documents.append(Document(
        page_content="""
        5G Network Architecture:

        The 5G System architecture consists of:
        - UE (User Equipment)
        - NG-RAN (Next Generation Radio Access Network)
        - 5GC (5G Core Network)

        Key Network Functions:
        - AMF (Access and Mobility Management Function)
        - SMF (Session Management Function)
        - UPF (User Plane Function)
        - PCF (Policy Control Function)
        - AUSF (Authentication Server Function)
        - UDM (Unified Data Management)
        - AF (Application Function)
        - NSSF (Network Slice Selection Function)

        Service-Based Architecture (SBA):
        - Network functions communicate through service-based interfaces
        - RESTful APIs for communication

        Network Slicing:
        - Multiple virtual networks on the same physical infrastructure
        - eMBB (Enhanced Mobile Broadband)
        - URLLC (Ultra-Reliable Low Latency Communications)
        - mMTC (Massive Machine Type Communications)
        """,
        metadata={"source": "sample_data", "title": "5G-Network-Architecture.doc"}
    ))

    return documents

In [ ]:
def prepare_3gpp_documents_with_metadata():
    """Enhanced version that preserves document structure and adds better metadata"""
    regular_docs = prepare_3gpp_documents()

    # Enhance metadata for each document
    enhanced_docs = []
    for doc in regular_docs:
        # Extract document type from title if possible
        doc_type = "general"
        title = doc.metadata.get("title", "")

        if "physical layer" in title.lower():
            doc_type = "physical_layer"
        elif "architecture" in title.lower():
            doc_type = "architecture"
        elif "key concepts" in title.lower():
            doc_type = "key_concepts"

        # Create enhanced metadata
        enhanced_metadata = {
            **doc.metadata,
            "doc_type": doc_type,
            "source_type": "3GPP"
        }

        enhanced_docs.append(Document(
            page_content=doc.page_content,
            metadata=enhanced_metadata
        ))

    return enhanced_docs

In [ ]:
# 2. Add training data as documents
def convert_training_data_to_documents(train_data):
    """Convert training data to Document format for RAG with improved metadata"""
    documents = []

    for i, item in enumerate(train_data):
        # Create a document from each QA pair
        content = f"Question: {item['question']}\n"

        # Add options
        for option_key in sorted(k for k in item if k.startswith("option ")):
            content += f"{option_key}: {item[option_key]}\n"

        content += f"Answer: {item['answer']}"

        content += "\n\n---\n\n"

        content += f"Category: {item['category']}"

        # Add explanation if available
        if 'explanation' in item:
            content += f"\nExplanation: {item['explanation']}"

        # Enhanced metadata
        metadata = {
            "source": "training_data",
            "id": i,   # Use category if available
        }

        documents.append(Document(
            page_content=content,
            metadata=metadata
        ))

    return documents

In [ ]:
# 3. Process documents for RAG
def prepare_rag_documents(documents):
    """Process documents for RAG with source-specific chunking strategy"""
    # Separate documents by source
    training_docs = [doc for doc in documents if doc.metadata.get("source") == "training_data"]
    technical_docs = [doc for doc in documents if doc.metadata.get("source") != "training_data"]

    # Don't chunk training data - keep QA pairs intact

    # Use larger chunks with more overlap for technical docs
    tech_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,  # Larger chunks for technical content
        chunk_overlap=50,  # More overlap to maintain context
        separators=["\n\n", "\n", ". ", " ", ""]
    )

    # Split only technical documents
    tech_chunks = tech_splitter.split_documents(technical_docs)

    # Combine unchunked training data with chunked technical docs
    all_chunks = training_docs + tech_chunks

    print(f"Kept {len(training_docs)} training documents intact")
    print(f"Split {len(technical_docs)} technical documents into {len(tech_chunks)} chunks")
    print(f"Total documents for vector store: {len(all_chunks)}")

    return all_chunks

In [ ]:
# 4. Vector Store
def create_vector_store(documents, api_key=None):
    """Create a vector store from documents using OpenAI embeddings"""
    print("Creating vector store...")

    # Initialize OpenAI embeddings
    embeddings = OpenAIEmbeddings(
        model="text-embedding-3-small",  # or use "text-embedding-ada-002", "text-embedding-3-small" or "text-embedding-3-large" for newer models
    )

    # Create vector store
    vectorstore = FAISS.from_documents(documents, embeddings)
    print("Vector store created successfully!")

    return vectorstore, embeddings

In [ ]:
# 4. define inference_with_rag()
def inference_with_rag(data, model_path, vectorstore, batch_size=32, k=4):
    """Use RAG-enhanced fine-tuned model with batching for inference"""
    import torch
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    import re

    print(f"Loading model from: {model_path}")

    # Use exactly the same loading approach as your fine-tuned version
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoPeftModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.float16,
        device_map="auto",
    )

    # Create retriever from vectorstore
    retriever = vectorstore.as_retriever(search_kwargs={"k": k})

    predictions = {}

    # Process in batches
    for i in range(0, len(data), batch_size):
        batch_data = data[i:i+batch_size]
        batch_prompts = []
        batch_ids = []

        for question_id, question_data in enumerate(batch_data, start=i):
            question = question_data['question']
            options_text = ""
            for option_key in sorted(k for k in question_data if k.startswith("option ")):
                options_text += f"{option_key}: {question_data[option_key]}\n"

            # Get relevant context from vector store for this question
            context_docs = retriever.get_relevant_documents(question)
            context = "\n\n".join([doc.page_content for doc in context_docs])

            # Create RAG-enhanced prompt that matches your fine-tuned style
            prompt = f"""You're a telemcommunications expert. Here's a multiple-choice question about telecommunications' {question_data['category'].lower() if question_data['category'] else ''}. Consider technical details carefully. Only respond with exactly one option without additional text, for example,

Example:
Question: What does HTTPS stand for?
option 1: Hypertext Transfer Protocol Secure
option 2: Hypertext Transfer Protocol Standard
option 3: Hypertext Transfer Protocol System
option 4: Hypertext Transfer Protocol Specification
option 5: Hypertext Transfer Protocol
Answer: option 1

Now, answer the question below in the same format:
Question: {question}
{options_text}[/INST]"""

            batch_prompts.append(prompt)
            batch_ids.append(question_id)

        # Tokenize batch
        inputs = tokenizer(batch_prompts, padding=True, padding_side='left', return_tensors="pt").to(model.device)

        # Generate for whole batch
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens=20,
                temperature=0.01,
                do_sample=False
            )

        # Process batch outputs
        for output, q_id, prompt in zip(outputs, batch_ids, batch_prompts):
            generated_text = tokenizer.decode(output, skip_special_tokens=True)

            # Remove the prompt from the generated text
            if prompt in generated_text:
                generated_text = generated_text[len(prompt):].strip().lower()
            else:
                generated_text = generated_text.strip().lower()

            # Extract answer
            match = re.search(r"option\s*(\d)", generated_text)
            if not match:
                match = re.search(r"^\D*(\d)\D*$", generated_text)

            if match:
                option_number = match.group(1)
                predictions[q_id] = f"option {option_number}"
            else:
                predictions[q_id] = "option 1"  # default fallback

    return predictions

In [ ]:
import time
# k=4
# 1. Prepare training data documents (improved version)
train_documents = convert_training_data_to_documents(train_data)

# 2. Prepare 3GPP documents (with enhanced metadata)
gpp_documents = prepare_3gpp_documents_with_metadata()

# 3. Process documents using your existing function
combined_docs = prepare_rag_documents(train_documents + gpp_documents)

# 4. Create vector store using your existing function
vectorstore, embeddings = create_vector_store(combined_docs)

# 5. Use the exact model path from your test code
model_path = "/content/telecom_finetuned/checkpoint-500"

# 6. Evaluate RAG-enhanced model
print("Evaluating RAG-enhanced model...")
start = time.time()
rag_predictions = inference_with_rag(val_data, model_path, vectorstore)
end = time.time()
rag_runtime = end - start
rag_accuracy = evaluate_model(val_data, rag_predictions)
print(f"RAG-enhanced model inference took {rag_runtime:.2f} seconds")
print(f"RAG-enhanced model accuracy: {rag_accuracy * 100:.2f}%")

{
    "rag_accuracy": rag_accuracy,
    "rag_runtime": rag_runtime,
}

Processing 3GPP documents...
Found documents directory at: /content/drive/MyDrive/airaChallenge/wireless_documents
Processed 10 documents so far...
Successfully processed 15 documents from /content/drive/MyDrive/airaChallenge/wireless_documents
Total processed documents: 15
Kept 6000 training documents intact
Split 15 technical documents into 15 chunks
Total documents for vector store: 6015
Creating vector store...


<ipython-input-6-16051422e34c>:8: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(


Vector store created successfully!
Evaluating RAG-enhanced model...
Loading model from: /content/telecom_finetuned/checkpoint-500


<ipython-input-7-7d2d66ffa744>:37: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  context_docs = retriever.get_relevant_documents(question)


RAG-enhanced model inference took 2164.16 seconds
RAG-enhanced model accuracy: 70.30%


{'rag_accuracy': 0.703, 'rag_runtime': 2164.1604523658752}

In [ ]:
import time
# k=7
# 1. Prepare training data documents (improved version)
train_documents = convert_training_data_to_documents(train_data)

# 2. Prepare 3GPP documents (with enhanced metadata)
gpp_documents = prepare_3gpp_documents_with_metadata()

# 3. Process documents using your existing function
combined_docs = prepare_rag_documents(train_documents + gpp_documents)

# 4. Create vector store using your existing function
vectorstore, embeddings = create_vector_store(combined_docs)

# 5. Use the exact model path from your test code
model_path = "/content/telecom_finetuned/checkpoint-500"

# 6. Evaluate RAG-enhanced model
print("Evaluating RAG-enhanced model...")
start = time.time()
rag_predictions = inference_with_rag(val_data, model_path, vectorstore, k=7)
end = time.time()
rag_runtime = end - start
rag_accuracy = evaluate_model(val_data, rag_predictions)
print(f"RAG-enhanced model inference took {rag_runtime:.2f} seconds")
print(f"RAG-enhanced model accuracy: {rag_accuracy * 100:.2f}%")

{
    "rag_accuracy": rag_accuracy,
    "rag_runtime": rag_runtime,
}

Processing 3GPP documents...
Found documents directory at: /content/drive/MyDrive/airaChallenge/wireless_documents
Processed 10 documents so far...
Successfully processed 15 documents from /content/drive/MyDrive/airaChallenge/wireless_documents
Total processed documents: 15
Kept 6000 training documents intact
Split 15 technical documents into 15 chunks
Total documents for vector store: 6015
Creating vector store...


<ipython-input-6-16051422e34c>:8: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(


Vector store created successfully!
Evaluating RAG-enhanced model...
Loading model from: /content/telecom_finetuned/checkpoint-500


<ipython-input-8-b319d1e474df>:37: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  context_docs = retriever.get_relevant_documents(question)


RAG-enhanced model inference took 2163.19 seconds
RAG-enhanced model accuracy: 70.30%


{'rag_accuracy': 0.703, 'rag_runtime': 2163.1861028671265}

In [ ]:
def implement_hybrid_solution():
    """Try both with and without 3GPP docs to compare performance"""
    import time
    import gc
    import torch

    # 1. Prepare training data documents (improved version)
    train_documents = convert_training_data_to_documents(train_data)

    # 2. Prepare 3GPP documents (with enhanced metadata)
    gpp_documents = prepare_3gpp_documents_with_metadata()

    # First test: Only training data
    print("\n--- Testing with training data only ---")
    train_only_docs = prepare_rag_documents(train_documents)
    train_only_vectorstore, _ = create_vector_store(train_only_docs)

    # Evaluate
    model_path = "/content/telecom_finetuned_model/checkpoint-400"
    start = time.time()
    train_only_predictions = inference_with_rag(val_data, model_path, train_only_vectorstore)
    train_only_runtime = time.time() - start
    train_only_accuracy = evaluate_model(val_data, train_only_predictions)

    del train_only_vectorstore
    gc.collect()
    torch.cuda.empty_cache()

    # Second test: Both training and 3GPP data
    print("\n--- Testing with training data + 3GPP documents ---")
    combined_docs = prepare_rag_documents(train_documents + gpp_documents)
    combined_vectorstore, _ = create_vector_store(combined_docs)

    # Evaluate
    start = time.time()
    combined_predictions = inference_with_rag(val_data, model_path, combined_vectorstore)
    combined_runtime = time.time() - start
    combined_accuracy = evaluate_model(val_data, combined_predictions)

    # Report results
    print("\n--- Results Comparison ---")
    print(f"Fine-tuned model only: {finetuned_accuracy * 100:.2f}%")
    print(f"RAG with training data only: {train_only_accuracy * 100:.2f}% (time: {train_only_runtime:.2f}s)")
    print(f"RAG with training + 3GPP data: {combined_accuracy * 100:.2f}% (time: {combined_runtime:.2f}s)")

    return {
        "train_only_accuracy": train_only_accuracy,
        "combined_accuracy": combined_accuracy,
        "train_only_runtime": train_only_runtime,
        "combined_runtime": combined_runtime
    }

In [ ]:
rag_solution = implement_hybrid_solution()

## Findings

### Summary Statistics
|                      |    naive |   finetuned |       rag |
|:---------------------|---------:|------------:|----------:|
| accuracy             |   0.6325 |      0.7225 |    0.703  |
| runtime  (in seconds)| 170.745  |   1248.2    |  2163.18  |

### Part A: Prompt Engineering and Inference Optimization

For the Basic Inference Engine task, I focused on optimizing inference performance while maintaining accuracy on the TeleQnA dataset. The key challenge was to efficiently serve the `Llama 3.2-3B instruct` model in a production-ready setup.

#### Model Serving Comparison

I implemented and compared two popular model serving frameworks:
1. `vLLM`: Emerged as the superior option for this task. Key advantages included:
  - Efficient batch processing (32 prompts per batch)
  - Optimized CUDA kernels for faster inference
  - Continuous batching that significantly reduced latency
  - Proper memory management with PagedAttention

2. `Ollama`: While easier to set up, it showed limitations:
  - Slower inference speeds despite multi-threading attempts
  - Less efficient memory management
  - Limited batch processing capabilities

#### Prompt Engineering

I experimented with several prompt configurations to improve accuracy:

- Implemented few-shot examples with telecommunications-specific content
- Added explicit instructions for output formatting (responding with only the option number)
- Included clear delimiters between question components
Used structured examples to demonstrate the desired answer format

The most effective prompt structure included:
```python
"""Answer the following multiple-choice question about telecommunications. Only respond with exactly one option (e.g., 'option 1', 'option 2', etc.) without any additional text.

Example 1:
Question: What is the capital of France?
option 1: Berlin
option 2: Madrid
option 3: Paris
option 4: Rome
option 5: Oslo
Answer: option 3

Example 2:
Question: What does HTTPS stand for?
option 1: Hypertext Transfer Protocol Secure
option 2: Hypertext Transfer Protocol Standard
option 3: Hypertext Transfer Protocol System
option 4: Hypertext Transfer Protocol Specification
option 5: Hypertext Transfer Protocol
Answer: option 1

Now, answer the question below in the same format:
Question: {question}
"""
```
#### Parallelization Strategies
To optimize runtime performance, I implemented:

- Batch processing of `32` questions at a time
- Asynchronous processing with proper concurrency management
- `ThreadPoolExecutor` for coordinating multiple inference requests
- Progress tracking with `tqdm` for monitoring batch processing
- `Regex`-based answer extraction for consistent response parsing

This parallelized approach with `vLLM` achieved an accuracy of `63.25%` with a runtime of just `170.7` seconds for the entire validation set.

---

### Part B: Fine-Tuning the LLM

#### Hyperparameter Optimization

I systematically explored various hyperparameters for LoRA fine-tuning:

**LoRA Configuration Experiments**
- Rank values: Tested `r=16` and `r=32`, with r=32 providing better performance
- Alpha settings: Set to 2x the rank value (`α=64` for `r=32`)
- Dropout rate: Tested `0.1`, `0.05`, and `0.01` as dropout rates. While `0.01` allowed for training time to be faster, it also led to overfitting to the training dataset, creating a considerable disparity between validation error and training error. I concluded that `0.05` was optimal as it balanced the tradeoff between runtime and performance of validation set.
- Target modules: Included query, key, value projection layers plus MLP components

#### Training Dynamics

- Step count analysis: Observed model convergence around 400-500 steps
- Overfitting detection: Monitored training vs. validation loss curves
- Epoch variation: Experimented with 3, 6, and 8 epochs, finding diminishing returns after 3
- Checkpoint evaluation: Implemented a function to evaluate all saved checkpoints to identify optimal model (`checkpoint-400`)

#### Error Analysis

**Category Error Rate**: Observed error rate by category to determine which categories are more likely to cause errors

| Category                | Error Rate | Notes  |
|:------------------------|------------|-------:|
| Standards overview      |   0.27     | Moderate difficulty with technical specs |
| Research publications   |   0.49     | Highest error rate - complex citations |
| Research overview       |   0.28     | Struggled with research methodology |
| Standards specifications|  0.37      | Technical details caused confusion |
| Lexicon                 |   0.13     | Lowest error rate - definitional content |

To address these category-specific errors, I modified the fine-tuning prompts to include category context:
```python
prompt = f"You're a telecommunications expert. Here's a multiple-choice question about telecommunications' {item['category'].lower() if item['category'] else ''}. Consider technical details carefully. Only respond with exactly one option without additional text."
```
This category-aware prompting significantly improved performance on the most challenging categories, particularly for Standards specifications and Research overview.

The fine-tuning process was executed using the `Unsloth` framework for efficiency, achieving a significant performance boost with an accuracy of `72.25%` on the validation set.

---

### Appendix: Retrieval-Augmented Generation (RAG) Implementation

#### Implementation Details and Architecture

I implemented a comprehensive RAG system to enhance the model's domain knowledge while maintaining inference efficiency:

1. **Document Processing Pipeline**:
  - Developed source-specific document processors for training data and 3GPP technical documents
  - Added enhanced metadata tagging to improve retrieval context awareness
  - Preserved document structure and relationships through intelligent chunking
2. **Embedding Strategy**:
  - Tested three OpenAI embedding models: `text-embedding-ada-002`, `text-embedding-3-small`, and `text-embedding-3-large`
  - Found `text-embedding-3-small` provided the optimal balance of performance and efficiency given the dataset size (`~3.9`MB)
  - Larger models like `text-embedding-3-large` did not provide sufficient accuracy gains to justify the increased computation time
3. **Retrieval Optimization**:
  - Implemented FAISS vector store for efficient similarity search
  - Extensive $k$-parameter testing revealed non-linear performance relationships
4. **Prompt Enhancement**: Integrated retrieved context into the prompt structure

#### Retrieval Parameter Experiments

The retriever's $k$ parameter (number of documents retrieved per query) had significant impact on model performance:

| k-Values| Accuracy| Runtime (s) | Notes   |
|:--------|---------|--------|-------------:|
| 3  |   0.6785     | 1873.44|Degraded performance vs. fine-tuned baseline|
| 4  |   0.703      | 2164.16|Optimal retrieval context size              |
| 7  |   0.703      | 2163.19|No additional gain with more documents      |
| 10 |  0.701       | 2560.29|Performance degradation with retrieval noise|


#### Chunking Strategy Analysis
A critical insight from my implementation was the importance of source-specific document chunking:

1. **Training Data**: Preserved QA pairs as atomic units to maintain the integrity of question-answer relationships:
```python
# Separate documents by source
# Keep training data intact - no chunking
training_docs = [doc for doc in documents if doc.metadata.get("source") == "training_data"]
# Don't chunk training data - keep QA pairs intact
```
2. **Technical Documents**: Implemented targeted chunking optimized for technical content:
```python
tech_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Optimized chunk size for technical content
    chunk_overlap=50,  # Sufficient overlap to maintain context
    separators=["\n\n", "\n", ". ", " ", ""]
)
```
This hybrid approach preserved the semantic structure of each document type, significantly improving retrieval relevance compared to a one-size-fits-all chunking strategy.

---

### Performance Comparision Analysis
Comparing the three approaches reveals important trade-offs:
1. **Basic Inference (Naive)**:
  - Fastest runtime (170.7s) but lowest accuracy (63.25%)
  - Excellent for quick responses but limited domain knowledge
2. **Fine-tuned Model**:
  - Best accuracy (72.25%) with moderate runtime (1248.2s)
  - Fixed knowledge embedded in model weights
3. **RAG Implementation**:
  - Good accuracy (70.3%) with longest runtime (2163.19s)
  - More flexible knowledge base that can be updated without retraining

The fine-tuned model outperformed RAG by approximately 2 percentage points while running nearly 2x faster. However, the RAG system offers key advantages in knowledge flexibility and extensibility that would be valuable in production scenarios where domain knowledge evolves rapidly.

---

### Conclusions and Production Recommendations

Based on these experiments, I recommend:

1. For static knowledge domains: Use the fine-tuned model for optimal performance-to-runtime ratio
2. For evolving domains: Use the RAG implementation with k=4 for balance of accuracy and efficiency
3. Consider a hybrid approach: Use the fine-tuned model as the base with RAG enhancement only for challenging questions or new domain knowledge areas

This tiered implementation strategy would maximize both accuracy and computational efficiency in a production environment.

---

### Future Considerations and Improvements

Based on the results and challenges encountered in this implementation, several promising avenues for improvement emerge:

1. **Hybrid Retrieval System**:
  - Implement a combination of dense and sparse retrievers (BM25 + embedding-based) to capture both keyword matching and semantic similarity
  - Add a re-ranking layer using cross-encoders to further refine the retrieval results before passing to the LLM
2. **Inference Optimization**:
  - Implement caching at multiple levels (embedding cache, retrieval cache, and response cache) to improve repeat question performance
  - Explore quantization methods like INT4/INT8 to further reduce latency without significant accuracy drops
3. **Adaptive Retrieval System**
  - Develop a confidence-based system that dynamically adjusts the retrieval parameters (k value) based on question difficulty
  - Implement query expansion techniques to improve retrieval for questions with limited context
4. **Knowledge Base Expansion**:
- Incorporate additional telecommunications standards beyond 3GPP
- Add specialized knowledge sources for categories where the model showed weaknesses

These improvements would provide promising directions for evolving this system into a more robust, accurate, and efficient production solution for telecommunications question answering.